In [7]:
import torch
import plotly.graph_objects as go
import numpy as np 


In [3]:
torch.cuda.is_available()

True

In [196]:
# Batch Size, Input Neurons, Hidden Neurons, Output Neurons
N, D_in, H, D_out, samples = 32, 1, 16, 1, 100

if N > samples:
    N=samples

# # Create random Tensors to hold inputs and outputs
# x = torch.randn(N, D_in)
# y = torch.randn(N, D_out)

# Create ground truth curve and noisy measurements

gt_x = np.linspace(-10, 10, samples, dtype='float32')
# gt_y = 1/(1+np.exp(-1*gt_x))
gt_y = np.sin(gt_x)

noise_x = np.random.normal(0, 0.1, gt_x.shape[0]).astype('f')
noise_y = np.random.normal(0, 0.1, gt_y.shape[0]).astype('f')
data_x = gt_x + noise_x
data_y = gt_y + noise_y

# Use the nn package to define our model
# Linear (Input -> Hidden), ReLU (Non-linearity), Linear (Hidden-> Output)
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# Define the loss function: Mean Squared Error
# The sum of the squares of the differences between prediction and ground truth
loss_fn = torch.nn.MSELoss(reduction='sum')

# The optimizer does a lot of the work of actually calculating gradients and
# applying backpropagation through the network to update weights
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

fig = go.Figure()
fig.add_trace(go.Scatter(x=data_x, y=data_y, mode="markers"))
fig.add_trace(go.Scatter(x=gt_x, y=gt_y, mode="lines"))



In [53]:
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

In [199]:
fig = go.Figure(
    data=[go.Scatter(x=gt_x, y=gt_y, mode="lines"), go.Scatter(x=data_x, y=data_y, mode="markers")],
    layout=go.Layout(
        xaxis=dict(range=[-10, 10], autorange=False, zeroline=False),
        yaxis=dict(range=[-2, 2], autorange=False, zeroline=False),
        title_text="Kinematic Generation of a Planar Curve", hovermode="closest",
        updatemenus=[dict(type="buttons",
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None, {"frame": {"duration": 1, "redraw": True},}])])]),
    frames=[go.Frame(
        data=[go.Scatter(
            x=np.linspace(-10, 10, 100),
            y=fits[k],
            mode="lines"), go.Scatter(x = samps_x[k], y = samps_y[k], mode='markers')])

        for k in range(fits.shape[0])]
)
fig.update_layout(autosize=False,
    width=1500,
    height=1500,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ), 
    transition ={'duration':0})
for i, frame in enumerate(fig.frames):
    frame.layout.title = "Epoch: {}".format(str(i*100))
fig.show()

In [178]:
samps_x[0]

array([8.63967], dtype=float32)

In [197]:
# Perform 30000 training steps
steps = 10000
ind_array =np.arange(samples)
fits = np.zeros((int(steps/100), 100))
samps_x = np.zeros((int(steps/100), N))
samps_y = np.zeros((int(steps/100), N))
for t in range(steps):
    # Forward pass: compute predicted y by passing x to the model.
    inds = np.random.choice(ind_array, size = N, replace=False)
    x = torch.from_numpy(data_x[inds, None])
    y = torch.from_numpy(data_y[inds, None])
    y_pred = model(x)

    # Compute loss and print it periodically
    loss = loss_fn(y_pred, y)
    if t % 1 == 0:
        print(t, loss.item())
        fits[int(t/100)] = model(torch.from_numpy(np.linspace(-10, 10, num=100)).reshape(-1, 1).float()).flatten().detach().numpy()
        samps_x[int(t/100)] = data_x[inds]
        samps_y[int(t/100)] = data_y[inds]
        

    # Update the network weights using gradient of the loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


0 35.294776916503906
1 23.51072883605957
2 21.86195182800293
3 26.25971221923828
4 25.354291915893555
5 20.383533477783203
6 18.405838012695312
7 15.176019668579102
8 20.473817825317383
9 19.61264419555664
10 21.994400024414062
11 14.19271469116211
12 20.605865478515625
13 16.66935920715332
14 20.891836166381836
15 19.973365783691406
16 19.9470157623291
17 15.806281089782715
18 16.24502182006836
19 14.279363632202148
20 17.576396942138672
21 13.646692276000977
22 17.435041427612305
23 16.759180068969727
24 19.863908767700195
25 20.700368881225586
26 18.84067726135254
27 18.077604293823242
28 17.6540584564209
29 14.49550724029541
30 16.600303649902344
31 17.86185073852539
32 18.851482391357422
33 16.006250381469727
34 16.21476173400879
35 14.283453941345215
36 17.07862663269043
37 13.533164024353027
38 13.706393241882324
39 18.24185562133789
40 17.735843658447266
41 11.565373420715332
42 15.32463264465332
43 12.799430847167969
44 16.207035064697266
45 15.37049388885498
46 15.34903144836

In [198]:
# Draw the original random points as a scatter plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_x, y=data_y, mode="markers"))

# Generate predictions for evnp.arange(samples)enly spaced x-values between minx and maxx
minx = min(list(gt_x))
maxx = max(list(gt_x))
c = torch.from_numpy(np.linspace(minx, maxx, num=640)).reshape(-1, 1).float()
d = model(c)

# Draw the predicted functions as a line graph
fig.add_trace(go.Scatter(x=c.flatten().numpy(), y=d.flatten().detach().numpy(), mode="lines"))
fig.add_trace(go.Scatter(x=gt_x, y=gt_y, mode="lines"))
fig.show()